In [ ]:
import vaex
import numpy as np
import pandas as pd
from glob import glob
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from scipy import interpolate

In [ ]:
# path to file produced by 1.8. Best RV.ipynb
file = <combined path>

In [ ]:
df = vaex.open(file)
df

In [ ]:
drop_cols = ['logg',
 'e_logg_lower',
 'e_logg_upper',
 'logg_symmetric',
 'logg_cat',
 'mh',
 'e_mh_lower',
 'e_mh_upper',
 'mh_symmetric',
 'mh_cat',
 'alphafe',
 'e_alphafe_lower',
 'e_alphafe_upper',
 'alphafe_symmetric',
 'alphafe_cat',
 'fem',
 'e_fem_lower',
 'e_fem_upper',
 'fem_symmetric',
 'fem_cat',
 'feh',
 'e_feh_lower',
 'e_feh_upper',
 'feh_symmetric',
 'feh_cat']
df.drop(drop_cols, inplace=True)
df

In [ ]:
np.pi

In [ ]:
# filter |z| < 200 pc and R < 150 pc
df_mask = df.filter("1/parallax * abs(sin(GLAT*3.141592653589793/180)) < 0.2").filter("1/parallax*cos(GLAT*3.141592653589793/180) < 0.15")
df_mask = df_mask.extract()
df_mask['M_J'] = df_mask.evaluate('Jmag -10 - 5*log10(1/parallax)')
df_mask['J-K'] = df_mask.evaluate('Jmag -Kmag')
df_mask

In [ ]:
# Optional! to check 3D distribution
xs = df_mask.evaluate("1/parallax*cos(GLAT*3.141592653589793/180)*cos(GLON*3.141592653589793/180)")

ys = df_mask.evaluate("1/parallax*cos(GLAT*3.141592653589793/180)*sin(GLON*3.141592653589793/180)")

zs = df_mask.evaluate("1/parallax*sin(GLAT*3.141592653589793/180)")
len(xs)

In [ ]:
%%time
# creating figure
fig = plt.figure(figsize=(10,10))
# fig = plt.figure()
ax = Axes3D(fig)
# creating the plot
# plot_geeks = ax.scatter(xs[:lim], ys[:lim], zs[:lim], color='green', s=0.1)
plot_geeks = ax.scatter(xs, ys, zs, color='green', s=0.01)
  
# setting title and labels
ax.set_title("3D plot")
ax.set_xlabel('x-axis')
ax.set_ylabel('y-axis')
ax.set_zlabel('z-axis')
  
# displaying the plot
plt.show()

## Jobovy convenience function, with modification

In [ ]:
def load_spectral_types():
    spt_path = "mamajek-spectral-class.hdf5"
    df_spt = vaex.open(spt_path)
    df_filtered = df_spt[["SpT", "M_J", "J-H", "H-Ks"]]
    mask = (df_filtered["M_J"] == df_filtered["M_J"])*(df_filtered["H-Ks"] == df_filtered["H-Ks"])*(df_filtered["J-H"] == df_filtered["J-H"])
    df_filtered_good = df_filtered[mask].to_pandas_df()
    df_filtered_good['J-K'] = df_filtered_good['J-H']+df_filtered_good['H-Ks']
    return df_filtered_good

sp= load_spectral_types()
sp_indx= np.array([(not 'O' in s)*(not 'L' in s)\
                          *(not 'T' in s)*(not 'Y' in s)\
                          *(not '.5V' in s) for s in sp['SpT']],
                     dtype='bool')
# Cut out the small part where the color decreases
sp_indx*= (np.roll((sp['J-K']),1)-(sp['J-K'])) <= 0.
ip_eems= interpolate.UnivariateSpline((sp['J-K'])[sp_indx],
                                      sp['M_J'][sp_indx],k=3,s=1.)
def main_sequence_cut_r(jk,low=False,tight=False):
    """Main-sequence cut, based on MJ, high as in low"""
    j_locus= ip_eems(jk)
    if low and tight:
        dj= 0.2-0.1*(j_locus-5.)
        dj[dj < 0.2]= 0.2
    elif low:
        dj= 0.2-0.25*(j_locus-5.)
        dj[dj < 0.2]= 0.2
    elif tight:
        djk= -(jk-0.6)/20.
        djk[djk>0.]= 0.
        j_locus= ip_eems(jk+djk)
        dj= 0.2-0.5*(j_locus-5.)
        dj[dj < 0.2]= 0.2
        dj[dj > 1.5]= 1.5
        dj*= -1.
    else:
        djk= -(jk-0.6)/5.
        djk[djk>0.]= 0.
        j_locus= ip_eems(jk+djk)
        dj= 1.-.8*(j_locus-5.)
        dj[dj < 0.2]= 0.2
        dj[dj > 2.5]= 2.5
        dj*= -1.
    return j_locus+dj

In [ ]:
# HR diagram
JK = np.linspace(0, 0.86, 100)
MJ = ip_eems(JK)
MJ_high = main_sequence_cut_r(JK, tight=True)
MJ_low = main_sequence_cut_r(JK, low=True)
plt.figure(figsize=(8,10))
# df_mask.plot("bp_rp", "Gmag -10 - 5*log10(1/parallax)", what=vaex.stat.count(), limits='95%')
df_mask.viz.scatter("J-K", "M_J", s=0.01, length_limit=500_000)
plt.plot(JK, MJ, c="red")
plt.plot(JK, MJ_low, c="green")
plt.plot(JK, MJ_high, c="green")
plt.xlabel("J-Ks")
plt.ylabel("M_J")
plt.xlim(-0, 1)
plt.ylim(0, 8)
plt.gca().invert_yaxis()
plt.show()

In [ ]:
# HR diagram
plt.figure(figsize=(8,10))
# df_mask.plot("bp_rp", "Gmag -10 - 5*log10(1/parallax)", what=vaex.stat.count(), limits='95%')
df_mask.viz.scatter("Jmag - Kmag", "Jmag -10 - 5*log10(1/parallax)", s=0.01, length_limit=500_000)
plt.xlabel("J-Ks")
plt.ylabel("M_J")
plt.xlim(-1, 2)
plt.gca().invert_yaxis()
plt.show()

In [ ]:
# for gaia filter
def load_spectral_types_gaia():
    spt_path = "mamajek-spectral-class.hdf5"
    df_spt = vaex.open(spt_path)
    df_filtered = df_spt[["SpT", "M_G", "Bp-Rp"]]
    mask = (df_filtered["M_G"] == df_filtered["M_G"])*(df_filtered["Bp-Rp"] == df_filtered["Bp-Rp"])
    df_filtered_good = df_filtered[mask].to_pandas_df()
    return df_filtered_good

sp = load_spectral_types_gaia()
sp_indx= np.array([(not 'O' in s)*(not 'L' in s)\
                          *(not 'T' in s)*(not 'Y' in s)\
                          *(not '.5V' in s) for s in sp['SpT']],
                     dtype='bool')
# Cut out the small part where the color decreases
sp_indx*= (np.roll((sp['Bp-Rp']),1)-(sp['Bp-Rp'])) <= 0.
ip_eems_gaia= interpolate.UnivariateSpline((sp['Bp-Rp'])[sp_indx],
                                      sp['M_G'][sp_indx],k=3,s=0.2)
def main_sequence_cut_r_gaia(br,low=False,tight=False):
    """Main-sequence cut, based on MJ, high as in low"""
    g_locus= ip_eems_gaia(br)
    if low and tight:
        dg= 0.2-0.1*(g_locus-5.)
        dg[dg < 0.2]= 0.2
    elif low:
        dg= 0.2-0.25*(g_locus-5.)
        dg[dg < 0.5]= 0.5
    elif tight:
        dbr= -(br-0.6)/20.
        dbr[dbr>0.]= 0.
        g_locus= ip_eems_gaia(br+dbr)
        dg= 0.2-0.5*(g_locus-5.)
        dg[dg < 0.2]= 0.2
        dg[dg > 1.5]= 1.5
        dg*= -1.
    else:
        dbr= -(br-0.6)/5.
        dbr[dbr>0.]= 0.
        g_locus= ip_eems_gaia(br+dbr)
        dg= 1.-.8*(g_locus-5.)
        dg[dg < 0.2]= 0.2
        dg[dg > 2.5]= 2.5
        dg*= -1.
    return g_locus+dg

In [ ]:
# HR diagram for gaia filter
BR = np.linspace(0, 3, 100)
MG = ip_eems_gaia(BR)
MG_high = main_sequence_cut_r_gaia(BR, tight=True)
MG_low = main_sequence_cut_r_gaia(BR, low=True)
plt.figure(figsize=(8,10))
# df_mask.plot("bp_rp", "Gmag -10 - 5*log10(1/parallax)", what=vaex.stat.count(), limits='95%')
df_mask.viz.scatter("bp_rp", "Gmag -10 - 5*log10(1/parallax)", s=0.01, length_limit=500_000)
plt.plot(BR, MG, c="red")
plt.plot(BR, MG_low, c="green")
plt.plot(BR, MG_high, c="green")
plt.xlabel("Bp-Rp")
plt.ylabel("M_G")
plt.xlim(-0, 3)
plt.ylim(0, 12)
plt.gca().invert_yaxis()
plt.show()

In [ ]:
df_s = load_spectral_types()
df_s

In [ ]:
df_mask['M_J_upper'] = main_sequence_cut_r(df_mask['J-K'].to_numpy(), tight=True)
df_mask['M_J_lower'] = main_sequence_cut_r(df_mask['J-K'].to_numpy(), low=True)
df_mask

In [ ]:
# Filter A star
JK_lower = (df_s[df_s['SpT'] == 'B9V']['J-K'].to_numpy()[0]+df_s[df_s['SpT'] == 'A0V']['J-K'].to_numpy()[0])/2
JK_upper = (df_s[df_s['SpT'] == 'A9V']['J-K'].to_numpy()[0]+df_s[df_s['SpT'] == 'F0V']['J-K'].to_numpy()[0])/2

mask_A = ((df_mask['J-K'] > JK_lower)
          *(df_mask['J-K'] < JK_upper)
          *(df_mask['M_J_lower'] > df_mask['M_J'])
          *(df_mask['M_J_upper'] < df_mask['M_J']))
df_A = df_mask[mask_A]
df_A

In [ ]:
df_A.export("A-type.hdf5", progress=True)

In [ ]:
mask_A2 = df_mask['spectraltype_esphs'] == 'A'
df_A2 = df_mask[mask_A2]
df_A2
df_A2.export("A-type-2.hdf5", progress=True)

In [ ]:
df_A2.export("A-type-2.hdf5", progress=True)

In [ ]:
# HR diagram for gaia filter
plt.figure(figsize=(8,7))
# df_mask.plot("bp_rp", "Gmag -10 - 5*log10(1/parallax)", what=vaex.stat.count(), limits='95%')
df_A.viz.scatter("J-K", "M_J", s=2, length_limit=500_000)
df_A2.viz.scatter("J-K", "M_J", s=2, length_limit=500_000)
plt.xlabel("J-K")
plt.ylabel("M_J")
plt.xlim(-0.02, 0.14)
plt.ylim(-0.5, 2.5)
plt.gca().invert_yaxis()
plt.show()

In [ ]:
# HR diagram for gaia filter
plt.figure(figsize=(8,7))
# df_mask.plot("bp_rp", "Gmag -10 - 5*log10(1/parallax)", what=vaex.stat.count(), limits='95%')
df_A.viz.scatter("J-K", "Jmag", s=1, length_limit=500_000)
# df_A2.viz.scatter("J-K", "M_J", s=0.5, length_limit=500_000)
plt.xlabel("J-K")
plt.ylabel("Jmag")
# plt.xlim(-0.02, 0.14)
# plt.ylim(-0.5, 2.5)
plt.gca().invert_yaxis()
plt.show()

In [ ]:
df_A['Jmag'].max()

In [ ]:
# Filter F star
JK_lower = (df_s[df_s['SpT'] == 'A9V']['J-K'].to_numpy()[0]+df_s[df_s['SpT'] == 'F0V']['J-K'].to_numpy()[0])/2
JK_upper = (df_s[df_s['SpT'] == 'F9V']['J-K'].to_numpy()[0]+df_s[df_s['SpT'] == 'G0V']['J-K'].to_numpy()[0])/2

mask_F = ((df_mask['J-K'] > JK_lower)
          *(df_mask['J-K'] < JK_upper)
          *(df_mask['M_J_lower'] > df_mask['M_J'])
          *(df_mask['M_J_upper'] < df_mask['M_J']))
df_F = df_mask[mask_F]
df_F

In [ ]:
df_F.export("F-type.hdf5", progress=True)

In [ ]:
mask_F2 = df_mask['spectraltype_esphs'] == 'F'
df_F2 = df_mask[mask_F2]
df_F2

In [ ]:
df_F2.export("F-type-2.hdf5", progress=True)

In [ ]:
# HR diagram
plt.figure(figsize=(8,10))
# df_mask.plot("bp_rp", "Gmag -10 - 5*log10(1/parallax)", what=vaex.stat.count(), limits='95%')
df_F.viz.scatter("J-K", "M_J", s=1, length_limit=500_000)
plt.xlabel("J-K")
plt.ylabel("M_J")
# plt.xlim(-0, 3)
# plt.ylim(0, 12)
plt.gca().invert_yaxis()
plt.show()

In [ ]:
# HR diagram for gaia filter
plt.figure(figsize=(8,10))
# df_mask.plot("bp_rp", "Gmag -10 - 5*log10(1/parallax)", what=vaex.stat.count(), limits='95%')
df_F.viz.scatter("J-K", "Jmag", s=1, length_limit=500_000)
plt.xlabel("J-K")
plt.ylabel("J")
# plt.xlim(-0, 3)
# plt.ylim(0, 12)
plt.gca().invert_yaxis()
plt.show()

In [ ]:
df_F['Jmag'].max()

In [ ]:
# Filter G star
JK_lower = (df_s[df_s['SpT'] == 'F9V']['J-K'].to_numpy()[0]+df_s[df_s['SpT'] == 'G0V']['J-K'].to_numpy()[0])/2
JK_upper = (df_s[df_s['SpT'] == 'G3V']['J-K'].to_numpy()[0]+df_s[df_s['SpT'] == 'G4V']['J-K'].to_numpy()[0])/2

mask_G = ((df_mask['J-K'] > JK_lower)
          *(df_mask['J-K'] < JK_upper)
          *(df_mask['M_J_lower'] > df_mask['M_J'])
          *(df_mask['M_J_upper'] < df_mask['M_J']))
df_G = df_mask[mask_G]
df_G

In [ ]:
# HR diagram for gaia filter
plt.figure(figsize=(8,10))
# df_mask.plot("bp_rp", "Gmag -10 - 5*log10(1/parallax)", what=vaex.stat.count(), limits='95%')
df_G.viz.scatter("J-K", "M_J", s=1, length_limit=500_000)
plt.xlabel("J-K")
plt.ylabel("M_J")
# plt.xlim(-0, 3)
# plt.ylim(0, 12)
plt.gca().invert_yaxis()
plt.show()

In [ ]:
# HR diagram for gaia filter
plt.figure(figsize=(8,10))
# df_mask.plot("bp_rp", "Gmag -10 - 5*log10(1/parallax)", what=vaex.stat.count(), limits='95%')
df_G.viz.scatter("J-K", "Jmag", s=1, length_limit=500_000)
plt.xlabel("J-K")
plt.ylabel("Jmag")
# plt.xlim(-0, 3)
# plt.ylim(0, 12)
plt.gca().invert_yaxis()
plt.show()

In [ ]:
df_G.export("G-type.hdf5", progress=True)